Datasets (massShootings.csv, accidentalDeath.csv, fatalPoliceShootings.csv, gunOwnershipByState2022.csv) cleaned and ready to load into MongoDB

In [114]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt

import pymongo
from pymongo import MongoClient

In [115]:
# Read massShootings Data File
massShootings = pd.read_csv("Resources/massShootings.csv")

massShootings.head()

Incident ID Incident Date         State   City Or County  \
0      2425199     27-Sep-22  Pennsylvania     Philadelphia   
1      2425393     27-Sep-22      New York  Corona (Queens)   
2      2423747     25-Sep-22  Pennsylvania       Harrisburg   
3      2423338     25-Sep-22       Indiana          Hammond   
4      2423538     24-Sep-22          Ohio         Columbus   

                     Address  # Killed  # Injured  Operations  
0             6498 Ridge Ave         1          4         NaN  
1   88th St and Atlantic Ave         0          4         NaN  
2  2300 block of Woodlawn St         0          4         NaN  
3           6217 Kennedy Ave         1          3         NaN  
4   E 5th Ave and Osborn Ave         1          3         NaN

In [116]:
#split year to a new column for analysis
massShootings['Year'] = pd.DatetimeIndex(massShootings['Incident Date']).year

#rename(for MongoDB) & get the columns needed
massShootings.rename(columns={'Incident Date':'Date','City Or County':'City_Or_County','# Killed':'Killed','# Injured':'Injured'}, inplace=True)

massShootings=massShootings[['Year','Date','State','City_Or_County','Killed','Injured']]

massShootings

Year       Date         State   City_Or_County  Killed  Injured
0     2022  27-Sep-22  Pennsylvania     Philadelphia       1        4
1     2022  27-Sep-22      New York  Corona (Queens)       0        4
2     2022  25-Sep-22  Pennsylvania       Harrisburg       0        4
3     2022  25-Sep-22       Indiana          Hammond       1        3
4     2022  24-Sep-22          Ohio         Columbus       1        3
...    ...        ...           ...              ...     ...      ...
1995  2019  17-Jul-19         Texas          Lubbock       1        3
1996  2019  16-Jul-19         Texas      San Antonio       0        4
1997  2019  15-Jul-19       Georgia          Atlanta       0        4
1998  2019  14-Jul-19      Illinois          Chicago       0        4
1999  2019  13-Jul-19      Illinois          Chicago       1        4

[2000 rows x 6 columns]

In [117]:
#massShootings_byyear=massShootings.groupby(['Year']).sum()['# Killed']

#massShootings_byyear

In [118]:
# Read accidentalDeath Data File
accidentalDeath = pd.read_csv("Resources/accidentalDeath.csv")

accidentalDeath.head()

Incident ID Incident Date       State     City Or County  \
0      2424304     25-Sep-22    Virginia            Suffolk   
1      2424013     25-Sep-22        Ohio  Saint Clairsville   
2      2424073     24-Sep-22   Wisconsin          Milwaukee   
3      2423209     24-Sep-22    Michigan   Sterling Heights   
4      2424598     23-Sep-22  California   South Lake Tahoe   

                        Address  # Killed  # Injured  Operations  
0          200 block of Pine St         1          0         NaN  
1               139 N Market St         1          0         NaN  
2       6900 block of N 43rd St         1          0         NaN  
3  43000 block of Frontenac Ave         1          0         NaN  
4        700 block of James Ave         1          0         NaN

In [119]:
#split year to a new column for analysis
accidentalDeath['Year'] = pd.DatetimeIndex(accidentalDeath['Incident Date']).year

#rename(for MongoDB) & get the columns needed
accidentalDeath.rename(columns={'Incident Date':'Date','City Or County':'City_Or_County','# Killed':'Killed','# Injured':'Injured'}, inplace=True)

accidentalDeath=accidentalDeath[['Year','Date','State','City_Or_County','Killed']]

accidentalDeath.head()

Year       Date       State     City_Or_County  Killed
0  2022  25-Sep-22    Virginia            Suffolk       1
1  2022  25-Sep-22        Ohio  Saint Clairsville       1
2  2022  24-Sep-22   Wisconsin          Milwaukee       1
3  2022  24-Sep-22    Michigan   Sterling Heights       1
4  2022  23-Sep-22  California   South Lake Tahoe       1

In [120]:
# Read fatalPoliceShootings Data File 
fatalPoliceShootings = pd.read_csv("Resources/fatalPoliceShootings.csv")

fatalPoliceShootings.head()

id                name        date   manner_of_death       armed   age  \
0   3          Tim Elliot  2015-01-02              shot         gun  53.0   
1   4    Lewis Lee Lembke  2015-01-02              shot         gun  47.0   
2   5  John Paul Quintero  2015-01-03  shot and Tasered     unarmed  23.0   
3   8     Matthew Hoffman  2015-01-04              shot  toy weapon  32.0   
4   9   Michael Rodriguez  2015-01-04              shot    nail gun  39.0   

  gender race           city state  signs_of_mental_illness threat_level  \
0      M    A        Shelton    WA                     True       attack   
1      M    W          Aloha    OR                    False       attack   
2      M    H        Wichita    KS                    False        other   
3      M    W  San Francisco    CA                     True       attack   
4      M    H          Evans    CO                    False       attack   

          flee  body_camera  longitude  latitude  is_geocoding_exact  
0  Not fleeing        False   -123.122    47.247                True  
1  Not fleeing        False   -122.892    45.487                True  
2  Not fleeing        False    -97.281    37.695                True  
3  Not fleeing        False   -122.422    37.763                True  
4  Not fleeing        False   -104.692    40.384                True

In [121]:
#split year to a new column for analysis
fatalPoliceShootings['Year'] = pd.DatetimeIndex(fatalPoliceShootings['date']).year
#get the date between 2018 to 2022
fatalPoliceShootings=fatalPoliceShootings.loc[fatalPoliceShootings['Year']>=2018]

#rename the state (abbreviation) column to get the full state name by merging with stateName.csv 
fatalPoliceShootings.rename(columns={'state':'Code'}, inplace=True)

fatalPoliceShootings.head() 

id                   name        date manner_of_death armed   age  \
2933  3275  Mark Steven Parkinson  2018-01-01            shot   gun  65.0   
2934  3288      Archie Lawhon III  2018-01-01            shot   gun  42.0   
2935  3289             Gary Johns  2018-01-01            shot   gun  65.0   
2936  3291    Shana Diane Musquiz  2018-01-02            shot   gun  30.0   
2937  3292          Ronald Elliot  2018-01-02            shot   gun  49.0   

     gender race       city Code  signs_of_mental_illness threat_level  \
2933      M    W  Rossville   GA                     True       attack   
2934      M    W    Romance   AR                    False       attack   
2935      M    W  Edgewater   FL                     True       attack   
2936      F    W  Camp Wood   TX                    False       attack   
2937      M    W      Ozark   AR                     True       attack   

             flee  body_camera  longitude  latitude  is_geocoding_exact  Year  
2933  Not fleeing        False    -85.312    34.939                True  2018  
2934  Not fleeing        False        NaN       NaN                True  2018  
2935  Not fleeing         True    -80.911    28.944                True  2018  
2936  Not fleeing        False   -100.011    29.670                True  2018  
2937  Not fleeing        False    -93.835    35.490                True  2018

In [122]:
# read state names data
stateName=pd.read_csv('Resources/states.csv')
stateName=stateName[['Code','State']]
stateName.head()

Code       State
0   AL     Alabama
1   AK      Alaska
2   AZ     Arizona
3   AR    Arkansas
4   CA  California

In [123]:
# merge fatalPoliceShootings and stateName datasets to get the full sate name
fatalPoliceShootings=pd.merge(fatalPoliceShootings, stateName, how='left', on=['Code','Code'])
fatalPoliceShootings.head()

id                   name        date manner_of_death armed   age gender  \
0  3275  Mark Steven Parkinson  2018-01-01            shot   gun  65.0      M   
1  3288      Archie Lawhon III  2018-01-01            shot   gun  42.0      M   
2  3289             Gary Johns  2018-01-01            shot   gun  65.0      M   
3  3291    Shana Diane Musquiz  2018-01-02            shot   gun  30.0      F   
4  3292          Ronald Elliot  2018-01-02            shot   gun  49.0      M   

  race       city Code  signs_of_mental_illness threat_level         flee  \
0    W  Rossville   GA                     True       attack  Not fleeing   
1    W    Romance   AR                    False       attack  Not fleeing   
2    W  Edgewater   FL                     True       attack  Not fleeing   
3    W  Camp Wood   TX                    False       attack  Not fleeing   
4    W      Ozark   AR                     True       attack  Not fleeing   

   body_camera  longitude  latitude  is_geocoding_exact  Year     State  
0        False    -85.312    34.939                True  2018   Georgia  
1        False        NaN       NaN                True  2018  Arkansas  
2         True    -80.911    28.944                True  2018   Florida  
3        False   -100.011    29.670                True  2018     Texas  
4        False    -93.835    35.490                True  2018  Arkansas

In [124]:
# get the columns needed for analysis
fatalPoliceShootings=fatalPoliceShootings[['Year','State','date','city','manner_of_death']]
fatalPoliceShootings=fatalPoliceShootings.sort_values(['Year'],ascending=False)
fatalPoliceShootings

Year         State        date                    city manner_of_death
4834  2022    Washington  2022-09-22                  Sequim            shot
4257  2022     Tennessee  2022-03-11               Covington            shot
4327  2022          Ohio  2022-04-01              Youngstown            shot
4326  2022  Pennsylvania  2022-03-31                 Lebanon            shot
4325  2022  South Dakota  2022-03-31             Sioux Falls            shot
...    ...           ...         ...                     ...             ...
657   2018     Louisiana  2018-08-07              Shreveport            shot
656   2018       Arizona  2018-08-07  Salt River Reservation            shot
655   2018    California  2018-08-07             Los Angeles            shot
654   2018       Indiana  2018-08-07           Monroe County            shot
0     2018       Georgia  2018-01-01               Rossville            shot

[4835 rows x 5 columns]

In [125]:
#fatalPoliceShootings_byyear=fatalPoliceShootings.groupby(['Year']).count()['manner_of_death']
#fatalPoliceShootings_byyear

In [155]:
# Read gunOwnership Data File
gunOwnership = pd.read_csv("Resources/gunOwnershipByState2022.csv")

gunOwnership.head()

State  gunOwnership  totalGuns
0        Montana         0.663      22133
1        Wyoming         0.662     132806
2         Alaska         0.645      15824
3          Idaho         0.601      49566
4  West Virginia         0.585      35264

In [156]:
# Making a Connection with MongoClient
client = pymongo.MongoClient('mongodb://localhost:27017')

# set up database
db = client["gun_violenceDB"]

# collection
col1= db["massShootings"]
col2=db["accidentalDeath"]
col3=db["fatalPoliceShootings"]
col4=db["gunOwnership"]

#load data to Mongodb
data = massShootings.to_dict(orient='records') 
col1.insert_many(data)

data = accidentalDeath.to_dict(orient='records') 
col2.insert_many(data)

data = fatalPoliceShootings.to_dict(orient='records') 
col3.insert_many(data)

data = gunOwnership.to_dict(orient='records') 
col4.insert_many(data)